In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [27]:
# Define parameters
asset_type = "stock"        # Example values: stock, etf, futures, crypto, index, fx
period = "full"             # Example values: full, month, week, day
timeframe = "1min"          # Example values: 1min, 5min, 30min, 1hour, 1day
adjustment = "adj_splitdiv"    # Example values: adj_split, adj_splitdiv, UNADJUSTED

#STOCK_DATA_DIR = "/home/ubuntu/ibis/data/frd-historical/stock/csv/1min/adj_splitdiv"
STOCK_DATA_DIR = f"/home/ubuntu/ibis/data/frd-historical/{asset_type}/csv/{timeframe}/{adjustment}"
os.listdir(STOCK_DATA_DIR)
ticker = 'RDFN'
stock_csv_filename = "_".join([ticker, period, timeframe, adjustment.replace("_", '')]) + ".txt"
fp = os.path.join(STOCK_DATA_DIR, ticker[0], stock_csv_filename)
print(stock_csv_filename)
print(fp)

RDFN_full_1min_adjsplitdiv.txt
/home/ubuntu/ibis/data/frd-historical/stock/csv/1min/adj_splitdiv/R/RDFN_full_1min_adjsplitdiv.txt


In [29]:
prices_df = pd.read_csv(
    fp,
    sep=",",
    names=['date', 'open', 'high', 'low', 'close', 'volume'],
    header=0,  # Assuming the first row is a header, if not set to None
    on_bad_lines='warn',  # Skip bad lines
    engine='python'  # Use the Python engine for more flexible error handling
)


prices_df['date'] = pd.to_datetime(prices_df['date'])
print(prices_df.shape)

# Convert 'date' column to datetime if it's not already
prices_df['date'] = pd.to_datetime(prices_df['date'])

# Extract the day as YYYY-MM-DD
prices_df['day'] = prices_df['date'].dt.date

# Extract the time as HH:MM:SS
prices_df['time'] = prices_df['date'].dt.time

# Calculate the time ID (minute of the day from 1 to 1440)
prices_df['time_id'] = prices_df['date'].dt.hour * 60 + prices_df['date'].dt.minute + 1

print(prices_df.info())
prices_df

(676378, 6)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 676378 entries, 0 to 676377
Data columns (total 9 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   date     676378 non-null  datetime64[ns]
 1   open     676378 non-null  float64       
 2   high     676378 non-null  float64       
 3   low      676378 non-null  float64       
 4   close    676378 non-null  float64       
 5   volume   676378 non-null  int64         
 6   day      676378 non-null  object        
 7   time     676378 non-null  object        
 8   time_id  676378 non-null  int32         
dtypes: datetime64[ns](1), float64(4), int32(1), int64(1), object(2)
memory usage: 43.9+ MB
None


,date,open,high,low,close,volume,day,time,time_id
0,2017-07-28 11:06:00,19.7100,20.0700,19.6300,19.9700,389751,2017-07-28,11:06:00,667
1,2017-07-28 11:07:00,19.9800,20.3600,19.9200,20.0800,218754,2017-07-28,11:07:00,668
2,2017-07-28 11:08:00,20.1500,20.6920,20.1250,20.6920,253592,2017-07-28,11:08:00,669
3,2017-07-28 11:09:00,20.6800,20.9200,20.2200,20.4000,125916,2017-07-28,11:09:00,670
4,2017-07-28 11:10:00,20.3400,20.4300,19.8000,19.9500,210876,2017-07-28,11:10:00,671
...,...,...,...,...,...,...,...,...,...
676373,2024-07-19 18:35:00,7.9200,7.9200,7.9200,7.9200,111,2024-07-19,18:35:00,1116
676374,2024-07-19 19:11:00,7.9200,7.9200,7.9200,7.9200,100,2024-07-19,19:11:00,1152
676375,2024-07-19 19:24:00,7.9191,7.9191,7.9191,7.9191,150,2024-07-19,19:24:00,1165
676376,2024-07-19 19:46:00,7.9000,7.9000,7.9000,7.9000,6375,2024-07-19,19:46:00,1187
